# AutoEncoders


In [1]:
import numpy as np
import pandas as pd
import torch
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
class StackedAE(nn.Module):
    """
        Description:
            creates a stacked auto encoder, builds off the
            nn.Module class
            
        Functions:
            __init__(self, num_movies, num_nodes_layer1, num_nodes_layer2)
            forward(X)
    """
    #def __init__(self, num_movies, num_nodes_layer1, num_nodes_layer2):
    def __init__(self,):
        """
            Description:
                initializes the autoencoder
            
            Params:
                num_movies:
                num_nodes_layer1: 
                num_nodes_layer2:
                
            Returns:
                none
        """
        super(StackedAE, self).__init__()
        num_movies = 1682
        num_nodes_layer1 = 20
        num_nodes_layer2 = 10
        self.fully_connected1 = nn.Linear(num_movies, num_nodes_layer1)
        self.fully_connected2 = nn.Linear(num_nodes_layer1, num_nodes_layer2)
        self.fully_connected3 = nn.Linear(num_nodes_layer2, num_nodes_layer1)
        self.fully_connected4 = nn.Linear(num_nodes_layer1, num_movies)
        
        self.activation = nn.Sigmoid()
        return
    
    def forward(self, X):
        """
            Description:
                propagates the vector through the network
                
            Params:
                X: the input vector
            
            Returns:
                X: the decoded vector (after transformation)
        """
        X = self.activation(self.fully_connected1(X))
        X = self.activation(self.fully_connected2(X))
        X = self.activation(self.fully_connected3(X))
        X = self.fully_connected4(X)
        return X
        

### get the dataset

In [3]:
movies = pd.read_csv('datasets/ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
# MovieID::Title::Genres
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users = pd.read_csv('datasets/ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
# UserID::Gender::Age::Occupation::Zip-code
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings = pd.read_csv('datasets/ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
# UserID::MovieID::Rating::Timestamp
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### train-test set prep

In [6]:
X_train = pd.read_csv('datasets/ml-100k/u1.base', delimiter='\t')
X_test  = pd.read_csv('datasets/ml-100k/u1.test', delimiter='\t')

X_train = np.array(X_train, dtype='int')
X_test  = np.array(X_test, dtype='int')

# UserID::MovieID::Rating::Timestamp
X_test[0:5]

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198],
       [        1,        20,         4, 887431883]])

In [7]:
user_count  = int(max(max(X_train[:,0]), max(X_test[:,0])))
movie_count = int(max(max(X_train[:,1]), max(X_test[:,1])))
print("user_count:  ", user_count)
print("movie_count: ", movie_count)

user_count:   943
movie_count:  1682


In [8]:
def convert(X, M, N):
    """
        converts into list of list for pytorch input
               : { movie 1, movie 2, ... , movie N}
        {
         user 1    rating   rating         rating
         user 2    rating   rating         rating
         ...
         user M    rating   rating         rating
         }
    """
    X_new = []
    
    for user_id in range(1, M+1):
        movies  = X[:,1][X[:,0] == user_id]
        ratings = X[:,2][X[:,0] == user_id]
        
        user_ratings = np.zeros(N)
        user_ratings[movies-1] = ratings
        X_new.append(list(user_ratings))
    
    return X_new

In [9]:
X_train  = convert(X_train, user_count, movie_count)
X_test   = convert(X_test, user_count, movie_count)

In [10]:
X_trainT = torch.FloatTensor(X_train)
X_testT  = torch.FloatTensor(X_test)

### Define the autoencoder

In [16]:
sae = StackedAE()

In [17]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=.01, weight_decay=.5)

### Train the autoencoder

In [20]:
epochs = 200

for epoch in range(1, epochs+1):
    train_loss = 0
    num_rated  = 0.0 # number of users who rate at least 1 movie
    for user in range(user_count):
        vector = Variable(X_trainT[user]).unsqueeze(0)
        target = vector.clone()
        if torch.sum(target.data > 0):
            output              = stackedAE(vector)
            target.require_grad = False
            output[target == 0] = 0
            vector_loss         = criterion(output, target)
            mean_corr           = movie_count/float(torch.sum(target.data > 0) + 1e-10)
            vector_loss.backward()
            train_loss          = train_loss + np.sqrt(vector_loss.item() * mean_corr)
            num_rated           = num_rated + 1.0
            optimizer.step()
    print("[{:02d}] training_loss: {:.3f}".format(epoch, train_loss))

[01] training_loss: 1670.026
[02] training_loss: 1034.193
[03] training_loss: 993.447


KeyboardInterrupt: 

In [ ]:
output

In [ ]:
print("{2:.0d}".format(2))